# 📓 The GenAI Revolution Cookbook

**Title:** Mastering Fine-Tuning of Large Language Models for Domain-Specific Success

**Description:** Unlock the potential of large language models with our step-by-step guide to fine-tuning for specialized applications, enhancing performance and achieving production-ready solutions.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



# Introduction

In this tutorial, we will walk through the process of fine-tuning a large language model (LLM) using the Hugging Face Transformers library. This guide is designed for AI Builders who are looking to deepen their expertise in deploying real-world applications with Generative AI. By the end of this tutorial, you will have a comprehensive understanding of how to fine-tune a model for domain-specific tasks, optimize it for production, and evaluate its performance. Fine-tuning LLMs is crucial for enhancing model accuracy and utility in specialized applications, such as creating a memory-aware chatbot or a document summarization assistant.

# Installation

To get started, we need to install the necessary libraries. These installations will provide you with the tools required to work with Hugging Face Transformers and other dependencies. For detailed documentation, visit the [Hugging Face Transformers documentation](https://huggingface.co/docs/transformers). For a more comprehensive guide on fine-tuning with these tools, see our [in-depth walkthrough on fine-tuning LLMs with Hugging Face](/blog/44830763/mastering-fine-tuning-of-large-language-models-with-hugging-face).

In [ ]:
# Install required libraries for LLM fine-tuning with Hugging Face ecosystem
# transformers: Provides pre-trained models and training utilities
!pip install transformers

# datasets: Enables easy loading and processing of training datasets
!pip install datasets

# torch: PyTorch backend required for model training and inference
!pip install torch

# Project Setup

Before diving into the code, ensure you have set up your environment variables and configuration files. This step is crucial for managing API keys, dataset paths, and other sensitive information securely.

# Step-by-Step Build

## Data Handling and Tokenization

First, we need to load and tokenize our dataset. This step involves preparing the data for training by converting text into a format that the model can understand.

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

def prepare_dataset(dataset_name, tokenizer, max_length=512):
    """
    Load and tokenize dataset for fine-tuning.
    
    Args:
        dataset_name (str): Name or path of the dataset to load
        tokenizer: Hugging Face tokenizer instance
        max_length (int): Maximum sequence length for tokenization
        
    Returns:
        tuple: Tokenized train and evaluation datasets
    """
    dataset = load_dataset(dataset_name)
    
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=max_length
        )
    
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    
    return tokenized_dataset["train"], tokenized_dataset["validation"]

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Prepare dataset
train_dataset, eval_dataset = prepare_dataset("wikitext", tokenizer)

## Model Integration and Training

Next, we integrate our pre-trained model and configure it for fine-tuning. This involves setting up training arguments and initiating the training process.

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import torch

def fine_tune_model(
    model_name="gpt2",
    dataset_name="wikitext",
    output_dir="./results",
    num_epochs=3,
    batch_size=4,
    learning_rate=5e-5
):
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    train_dataset, eval_dataset = prepare_dataset(dataset_name, tokenizer)
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        evaluation_strategy="steps",
        eval_steps=500,
        logging_dir=f"{output_dir}/logs",
        logging_steps=100,
        save_steps=1000,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        seed=42
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer
    )
    
    trainer.train()
    trainer.save_model(f"{output_dir}/final_model")
    tokenizer.save_pretrained(f"{output_dir}/final_model")
    
    return trainer

# Fine-tune model
trainer = fine_tune_model()

## Full End-to-End Application

Now, let's put all the components together in a single, runnable script. This script will demonstrate a working example that you can replicate.

In [ ]:
if __name__ == "__main__":
    try:
        trainer = fine_tune_model(
            model_name="gpt2",
            dataset_name="wikitext",
            output_dir="./fine_tuned_model",
            num_epochs=3,
            batch_size=4
        )
        print("Fine-tuning completed successfully!")
        
    except Exception as e:
        print(f"Fine-tuning failed with error: {str(e)}")

# Testing & Validation

Testing and validation are crucial to ensure the model's performance and reliability. Here, we show test runs and evaluation results to validate the fine-tuned model.

In [ ]:
# Example test run
sample_input = "Once upon a time"
input_ids = tokenizer.encode(sample_input, return_tensors="pt")
output = model.generate(input_ids, max_length=50, num_return_sequences=1)
print(tokenizer.decode(output[0], skip_special_tokens=True))

# Conclusion

In this tutorial, we explored the process of fine-tuning a large language model using Hugging Face Transformers. We covered data handling, model integration, and training, culminating in a complete, runnable application. Fine-tuning LLMs can significantly enhance their performance for domain-specific tasks, making them more effective and useful in real-world applications. For further strategies and techniques, explore our [detailed guide on mastering fine-tuning techniques](/blog/44830763/mastering-fine-tuning-of-large-language-models-with-hugging-face).